In [1]:
conda env list

# conda environments:
#
base                     /localtmp/ktm8eh/miniconda3
MMEDIT                *  /localtmp/ktm8eh/miniconda3/envs/MMEDIT
RL                       /localtmp/ktm8eh/miniconda3/envs/RL
TableText                /localtmp/ktm8eh/miniconda3/envs/TableText
VAD                      /localtmp/ktm8eh/miniconda3/envs/VAD
dassl                    /localtmp/ktm8eh/miniconda3/envs/dassl
editing                  /localtmp/ktm8eh/miniconda3/envs/editing


Note: you may need to restart the kernel to use updated packages.


In [1]:
!python ./vqa_test.py

loading VQA annotations and questions into memory...
creating index...
index created!
Question: What building is this?
Answer 1: church
Answer 2: church
Answer 3: church
Answer 4: church
Answer 5: church
Answer 6: church
Answer 7: church
Answer 8: church
Answer 9: church
Answer 10: church
Figure(640x480)
Loading VIT
Position interpolate from 16x16 to 26x26
freeze vision encoder
Loading VIT Done
Loading Q-Former
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'bert.embeddings.token_type_embeddings.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (

In [1]:
import transformers
import sys
sys.path.append("../easyeditor/dataset/processor/")
sys.path.append("../")
from blip_processors import BlipImageEvalProcessor
from easyeditor import MiniGPT4
from vqa import VQA
import random
import matplotlib.pyplot as plt
import os
import torch
from PIL import Image

In [2]:
from easyeditor import MiniGPT4
qformer_checkpoint= "../hugging_cache/blip2_pretrained_flant5xxl.pth"
qformer_name_or_path= "bert-base-uncased"
state_dict_file= "../hugging_cache/eva_vit_g.pth"
name= "../hugging_cache/vicuna-7b"
model_name= "minigpt4"
model = MiniGPT4(
    vit_model="eva_clip_g",
    q_former_model=qformer_checkpoint,
    img_size=364,
    use_grad_checkpoint=True,
    vit_precision="fp32",
    freeze_vit=True,
    llama_model=name,
    state_dict_file=state_dict_file,
    qformer_name_or_path=qformer_name_or_path
)

Loading VIT
Position interpolate from 16x16 to 26x26
freeze vision encoder
Loading VIT Done
Loading Q-Former


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['bert.embeddings.token_type_embeddings.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.3.output_query.LayerNorm.weight', 'bert.encoder.layer.2.intermediate_query.dense.weight', 'bert.encoder.layer.9.output_query.LayerNorm.bias', 'bert.encoder.layer.2.crossat

load checkpoint from ../hugging_cache/blip2_pretrained_flant5xxl.pth
freeze Qformer
Loading Q-Former Done
Loading LLAMA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LLAMA Done


In [3]:
model=model.to('cuda')

In [4]:
ROOT		='/localtmp/ktm8eh/datasets/VQA/okvqa'
# versionType ='v2_' # this should be '' when using VQA v2.0 dataset
taskType    ='OpenEnded' # 'OpenEnded' only for v2.0. 'OpenEnded' or 'MultipleChoice' for v1.0
dataSubType ='val2014'
annfile = os.path.join(ROOT, 'mscoco_val2014_annotations.json')
quesfile = os.path.join(ROOT, 'OpenEnded_mscoco_val2014_questions.json')
imgDir 		= os.path.join("/localtmp/ktm8eh/datasets/mscoco/images", dataSubType)
# initialize VQA api for QA annotations
vqa=VQA(annotation_file=annfile, question_file=quesfile)

# load and display QA annotations for given question types
"""
All possible quesTypes for abstract and mscoco has been provided in respective text files in ../QuestionTypes/ folder.
"""
annIds = vqa.getQuesIds(quesTypes='six')
anns = vqa.loadQA(annIds)
randomAnn = random.choice(anns)
vqa.showQA([randomAnn])
imgId = randomAnn['image_id']
imgFilename = 'COCO_' + dataSubType + '_'+ str(imgId).zfill(12) + '.jpg'


loading VQA annotations and questions into memory...
creating index...
index created!
Question: Name the type of house where these windows are seen?
Answer 1: jail
Answer 2: jail
Answer 3: jail
Answer 4: jail
Answer 5: jail
Answer 6: jail
Answer 7: beach house
Answer 8: beach house
Answer 9: old
Answer 10: old


In [5]:
vis_processor = BlipImageEvalProcessor(image_size=364, mean=None, std=None)


In [6]:
question = ["Which city is this photo taken?"]
raw_image = Image.open(os.path.join(imgDir, imgFilename)).convert("RGB")
image = vis_processor(raw_image).unsqueeze(0).to('cuda')

In [7]:
ans = model.predict_answers(samples={"image": image, "text_input": question}, inference_method="generate")
print(ans)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/localtmp/ktm8eh/miniconda3/envs/MMEDIT/lib/python3.9/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


['average [unused333] [unused692] [unused305] [unused273]वডளต']


In [8]:
model.device()

In [1]:
import json

In [5]:
def load_prompt(prompt_file='mscoco_val2014_annotations.json', partition=0):
    with open(prompt_file, 'r') as file:
        prompt = json.load(file)
    anns = prompt['annotations']
    return anns

In [6]:
anns = load_prompt()

In [7]:
anns[0]['image_id']

297147

In [8]:
len(anns)

5046

In [9]:
image_ids = [anns[i]['image_id'] for i in range(len(anns))]

In [12]:
import collections
print([item for item, count in collections.Counter(image_ids).items() if count > 1])


[572399, 236542, 58472, 191585, 318671, 84643, 536375, 553942, 282037, 519475, 558253, 382307, 55299]


In [13]:
# find id is 572399
ann_ids = [ann for ann in anns if ann['image_id'] == 572399]

In [16]:
ann_ids

[{'image_id': 572399,
  'answer_type': 'other',
  'question_type': 'seven',
  'question_id': 5723996,
  'answers': [{'answer_id': 1,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 2,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 3,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 4,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 5,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 6,
    'raw_answer': 'man',
    'answer_confidence': 'yes',
    'answer': 'man'},
   {'answer_id': 7,
    'raw_answer': 'men',
    'answer_confidence': 'yes',
    'answer': 'men'},
   {'answer_id': 8,
    'raw_answer': 'men',
    'answer_confidence': 'yes',
    'answer': 'men'},
   {'answer_id': 9,
    'raw_answer': 'men',
    'answer_confidence': 'yes',
    'answer': 'men

In [18]:
question_ids = [anns[i]['question_id'] for i in range(len(anns))]

In [19]:
len(set(question_ids))

5046

In [21]:
anns_train = load_prompt(prompt_file='mscoco_train2014_annotations.json', partition=0)

In [22]:
question_ids_train = [anns_train[i]['question_id'] for i in range(len(anns_train))]

In [23]:
len(set(question_ids_train)), len(question_ids_train)

(9009, 9009)

In [26]:
questions = question_ids + question_ids_train

In [28]:
len(set(questions)), len(questions)

(14055, 14055)